## Connection Code

In [1]:
import serial
import sys
import glob
import time
import serial.tools.list_ports
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import clear_output

ser = None

def serial_ports():
     result = []
     ports = serial.tools.list_ports.comports()
     for port, desc, hwid in sorted(ports):
        comm =  "{}: {}".format(port, desc)
        result.append(comm) 
     return result

def InitSerial(port, bps = 9600, to = 0):
    global ser
    try:
        ser = serial.Serial(port, bps, timeout = to)  # open serial port
        ser.flushInput()
        ser.flushOutput()
        return ser.name
    except Exception as e:
        return 'ERR: ' + str(e)

def CloseSerial():
    return('done')
    try:
        ser.flush()
        ser.close()
        return 'done'
    except Exception as e:
        return 'ERR: ' + str(e)  


def WriteSerial(string):
    try:
        reply = ser.write(string.encode())
        return str(reply)
    except Exception as e:
        return 'ERR: ' + str(e)    

def ReadSerial():
    try:
        reply = ''
        while ser.in_waiting:
            reply = reply + ser.readline().decode()
        return reply
    except Exception as e:
        return 'ERR: ' +  str(e)
    
def SendRead(string):
    if WriteSerial(string):
        time.sleep(0.1)
        return ReadSerial()
    
updatedPortList = []

def search_for_ports():
    updatedPortList = []
    serialPortList = serial_ports()
    for x in serialPortList:
        updatedPortList.append(x[:x.index(':')])
    return updatedPortList

def on_value_change(change):
    with output2:
        global SerialPort 
        SerialPort = change['new']
        InitSerial(change['new'],115200)
        WriteSerial('\x03')
        run_some_code()

def on_value_change2(change):
    with output2:
        global SerialPort 
        SerialPort = change['new']
        InitSerial(change['new'],115200)
        WriteSerial('\x03')
        connect_bicycle_code()
                
def run_some_code():
    code = '''
import hub,utime
    '''
    WriteSerial('\x05')
    WriteSerial(code)
    WriteSerial('\x04')
    WriteSerial('\x03')
    
    time.sleep(1) #wait for everything to get over there and read all the replies
    words = ReadSerial()
#     print(words)
    if (">>>" in words):
        print("You are now connected to SPIKE Prime!")
    else:
        print("Error, try reconnecting or connecting to a different serial port.")

def connect_bicycle_code():
    connectCode = '''
import hub,utime
battery.info()
    '''
    motorTest = '''
import hub,utime
hub.port.D.info()
    '''
    WriteSerial('\x05')
    WriteSerial(connectCode)
    WriteSerial('\x04')
    WriteSerial('\x03')
    
    time.sleep(1) #wait for everything to get over there and read all the replies
    words = ReadSerial()
#     print(words)
    if (">>>" in words):
        WriteSerial('\x05')
        WriteSerial(motorTest)
        WriteSerial('\x04')
        time.sleep(1) #wait for everything to get over there and read all the replies
        motorwords = ReadSerial()
#         print (motorwords)
        if ('{\'type\': None}' not in motorwords):
            print("You are now connected to SPIKE Prime!")
        else:
            print("Error, are you sure the motor is plugged in to port D?")    
    else:
        print("Error, try reconnecting or connecting to a different serial port.")

## Functions

In [2]:
def move(port,rotations,speed):
    moveCode = """hub.port."""+str(port)+""".motor.run_for_degrees("""+str(360*rotations)+""", """+str(speed)+""")"""
    WriteSerial('\x05')
    WriteSerial(moveCode)
    WriteSerial('\x04')
    
def buttonPressed(buttonType): # only parameter is the button ("left" or "right")
    if(buttonType=="right"):
        buttonCode = """print(hub.button.right.was_pressed())"""
    elif(buttonType=="left"):
        buttonCode = """print(hub.button.left.was_pressed())"""
    WriteSerial('\x05')
    WriteSerial(buttonCode)
    WriteSerial('\x04')
    responses = ReadSerial()
    if("True" in responses):
        return True
    else:
        return False
    
from IPython.display import Markdown
def read():
    string1=''
    string2 = '  '
    while len(string2):
        time.sleep(0.1)
        string2 = ReadSerial()
        string1 = string1+string2
    return string1

def setupLightSensor(port):
    WriteSerial('import hub\r\n')
    WriteSerial('light_sensor = hub.port.'+str(port)+'.device\r\n')

def color(num):
    WriteSerial('print(light_sensor.get()['+str(num)+'])\r\n')
    try:
        return(int(read().split('\r\n')[-2]))
    except:
        print("Are you sure the light sensor is connected correctly?")

def getColors():
    return ((color(2),color(3),color(4)))

def getRGB():
    return(int(color(2)/1024*255),int(color(3)/1024*255),int(color(4)/1024*255))

def printColor(rgb):
    colors = ["#"+('%02x%02x%02x' %(rgb[0],rgb[1],rgb[2]))]
    display(Markdown('<br>'.join(
        f'<span style="font-family: monospace">{color} <span style="color: {color}">████████</span></span>'
        for color in colors
    )))
    return colors[0][1:]

def showPixel(x,y):
    pixelCode = """hub.display.pixel("""+str(x)+""","""+str(y)+""",9)"""
    WriteSerial('\x05')
    WriteSerial(pixelCode)
    WriteSerial('\x04')

def clearDisplay():
    clearCode = """hub.display.clear()"""
    WriteSerial('\x05')
    WriteSerial(clearCode)
    WriteSerial('\x04')
    
def getGyroData():
    functions = """hub.motion.gyroscope()"""
    WriteSerial('\x05')
    WriteSerial(functions)
    WriteSerial('\x04')
    time.sleep(0.05)
    response = ReadSerial()
    return [eval(i) for i in response.split("\n")[-2][1:-2].split(", ")]

def beep(frequency,length):
    functions = """hub.sound.beep("""+str(frequency)+""", """+str(length*1000)+""", 3)"""
    WriteSerial('\x05')
    WriteSerial(functions)
    WriteSerial('\x04')
    
def setupForceSensor(port):
    WriteSerial('import hub\r\n')
    WriteSerial('force_sensor = hub.port.'+str(port)+'.device\r\n')
    
def getForce():
    WriteSerial('print(force_sensor.get()[0])\r\n')
#     print(read())
    try:
        return(int(read().split('\r\n')[-2]))
    except:
        print("Are you sure the force sensor is connected correctly?")

def getDistance():
    WriteSerial('print(distance_sensor.get()[0])\r\n')
    words = read()
    try:
        return(int(words.split('\r\n')[-2]))
    except:
        print("Are you sure the distance sensor is connected correctly?")
        
def setupDistanceSensor(port):
    WriteSerial('import hub\r\n')
    WriteSerial('distance_sensor = hub.port.'+str(port)+'.device\r\n')